### COMP258 - NEURAL NETWORKS
### Nestor Romero - 301133331
### Assignment 2

In [11]:
#Basic libraries imports

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

%matplotlib inline

In [12]:
#Load data and perform basic data exploration

iris_data = pd.read_csv('iris1.csv', header=0, names=['sepl', 'sepw', 'petl', 'petw', 'class'])

print(iris_data.head(5))
print()
print(iris_data.describe())
print()
print(iris_data.info())
print()
print(iris_data.shape)
print()
print(iris_data['class'].value_counts())

#3 classes 50 samples per class (except setosa)

   sepl  sepw  petl  petw        class
0   4.9   3.0   1.4   0.2  Iris-setosa
1   4.7   3.2   1.3   0.2  Iris-setosa
2   4.6   3.1   1.5   0.2  Iris-setosa
3   5.0   3.6   1.4   0.2  Iris-setosa
4   5.4   3.9   1.7   0.4  Iris-setosa

             sepl        sepw        petl        petw
count  149.000000  149.000000  149.000000  149.000000
mean     5.848322    3.051007    3.774497    1.205369
std      0.828594    0.433499    1.759651    0.761292
min      4.300000    2.000000    1.000000    0.100000
25%      5.100000    2.800000    1.600000    0.300000
50%      5.800000    3.000000    4.400000    1.300000
75%      6.400000    3.300000    5.100000    1.800000
max      7.900000    4.400000    6.900000    2.500000

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sepl    149 non-null    float64
 1   sepw    149 non-null    float64
 2   petl    149 non-null 

In [13]:
#create output values according to labels in data
iris_data['out_versicolor'] = iris_data['class'].apply( lambda x: 1 if x == 'Iris-versicolor' else 0)
iris_data['out_virginica'] = iris_data['class'].apply( lambda x: 1 if x == 'Iris-virginica' else 0)
iris_data['out_setosa'] = iris_data['class'].apply( lambda x: 1 if x == 'Iris-setosa' else 0)

iris_data.head(5)

#versicolor validation
iris_data[iris_data['class']=='Iris-versicolor']

#remove unnecessary class column
iris_data_prep = iris_data.drop(labels='class', axis=1)
iris_data_prep.head(2)

,sepl,sepw,petl,petw,out_versicolor,out_virginica,out_setosa
0,4.9,3.0,1.4,0.2,0,0,1
1,4.7,3.2,1.3,0.2,0,0,1


In [14]:
#function to create random weights to initialize network
def create_weights( num_units, num_inputs):
    '''
    Create a matrix of random values with shape (num_units, num_inputs)
    num_units = Number of neurons in layer
    num_inputs = Number of inputs from previous layer
    '''
#     weights = 2* np.array(np.random.rand(num_units, num_inputs)) - 1
    weights = np.array(np.random.rand(num_units, num_inputs))
    return weights

# sigmoid function
sigmoid = lambda x: 1/(1 + np.exp(-x));

# function to format results selecting the biggest value
def format_results( results_raw ):
    '''
    Creates a results array by selecting the maximun value per row 
    from the raw results obtained from the NN
    '''
    results = np.zeros(shape=results_raw.shape)
    for i in range(0,results_raw.shape[0]):
        results[i][np.argmax(results_raw[i])] = 1
    
    return results

In [15]:
# function to execute final feed forward step with the trained network
def feed_forward(whl,wol, inputs, outputs):
    '''
    This function calculates only the feed forward step for the network
    It is used to calculate the final results of the network
    whl = weights hidden layer
    wol = weights output layer
    '''
    num_records = inputs.shape[0]
    results = a = np.zeros(outputs.shape)
    
    for i in range(0,num_records):
        
        #for each row of values and results
        x = inputs[i, :].T;    # current set of values
        d = outputs[i];        # current set of results
        
        # FEEDFORWARD SECTION
        # HIDDEN LAYER
        # Calculate weighted sum and output
        v1 = np.dot(whl,x);
        y1 = sigmoid(v1);
        
        # OUTPUT LAYER
        # Calculate weighted sum and output
        v = np.dot(wol,y1);
        y = sigmoid(v);
        
        #create results
        print(f'd = {d} y = {y}')
        results[i] = y
        
    return results

In [16]:
# function to execute backpropagation algorithm
def back_propagation(whl, wol, inputs, outputs):
    '''
    This function calculates the feed forward and back propagation steps for the network
    whl = weights hidden layer
    wol = weights output layer
    '''
    # learning rate
    alpha = 0.9;
    
    ### size variables to avoid hardcoding
    num_records = inputs.shape[0]
    size_input = inputs.shape[1]
    size_output = outputs.shape[1]
    units_hidden = whl.shape[0]
    units_output = wol.shape[0]
    ###
    
    for i in range(0,num_records):
        
#         print(f'\n\nCURRENT ROW: {i}')
        
        #for each row of values and results
        x = inputs[i, :].T;    # current set of values
        d = outputs[i];        # current set of results
        
        # FEEDFORWARD SECTION
        # HIDDEN LAYER
        # Calculate weighted sum and output
        v1 = np.dot(whl,x);
        y1 = sigmoid(v1);
#         print(f'v1: {v1}')
#         print(f'y1: {y1}')
        
        # OUTPUT LAYER
        # Calculate weighted sum and output
        v = np.dot(wol,y1);
        y = sigmoid(v);
#         print(f'v: {v}')
#         print(f'y: {y}')
        
        # CALCULATE ERROR AND DELTA
        error = d - y;
        delta = y * (1 - y) * error; # sigmoid prime
        
#         print(f'delta: {delta.shape}')
        
        #
        # BACKPROPAGATION SECTION
        e1 = np.dot(wol.T, delta);
        delta1 = y1 * (1 - y1) * e1;  
        
#         print(f'error: {e1}')
#         print(f'delta1: {delta1}')
        
        # Adjust the weights according to the learning rule
        # HIDDEN LAYER
        
        delta1.shape=(units_hidden,1)  # column vector of deltas for the hidden layer
        x.shape=(1,size_input)  # row vector of the current input
        
        dwhl = alpha * np.dot(delta1,x);
        whl = whl + dwhl;
        
        # OUTPUT LAYER
        delta.shape = (size_output,1)
        y1.shape = (1, units_hidden)
        
        dwol = alpha*np.dot(delta,y1);
        wol = wol + dwol;
        
#         print(f'wol: {wol.shape}')
        
    return whl, wol;

In [17]:
# CREATE INPUT AND OUTPUT ARRAYS

inputs = iris_data_prep.iloc[:,0:4].to_numpy() #Input values (n,4)
outputs = iris_data_prep.iloc[:,4:].to_numpy() #Output values (n,3)

#Create train and test sets (70%-30% split)
inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(
    inputs, outputs, test_size=0.3, random_state=301133331)

#outputs_train

In [18]:
# MAIN PROGRAM EXECUTION
num_ephocs = 20

# Create initial weights
whl = create_weights(5,4)
wol = create_weights(3,5)

print('###\tINITIALIZE NETWORK WEIGHTS')
print('\tHidden Layer Weights')
print(whl)
print()
print('\tOutput Layer Weights')
print(wol)

print('\n###\tNETWORK ARCHITECTURE')
print(f'Weights Hidden Layer Shape {whl.shape}')
print(f'Weights Output Layer Shape {wol.shape}')

print()
print('###\tTRAINING NETWORK - ADJUST WEIGHTS')
print(f'Train Inputs Shape {inputs_train.shape}')
print(f'Train Outputs Shape {outputs_train.shape}')
print(f'Testing will proceed with {num_ephocs} ephocs!!')

# Use back propagation to obtain NN weights
for ei in range(1,num_ephocs+1):
    print(f'\tEPOCH: {ei}')
    whl, wol = back_propagation(whl, wol, inputs_train, outputs_train)



###	INITIALIZE NETWORK WEIGHTS
	Hidden Layer Weights
[[0.26281426 0.04814698 0.02547876 0.7358549 ]
 [0.42823608 0.89642488 0.00789098 0.91225424]
 [0.70578406 0.32243578 0.89301354 0.92424831]
 [0.76807199 0.55177746 0.54372977 0.40355329]
 [0.30091842 0.78399489 0.49696355 0.97345876]]

	Output Layer Weights
[[0.84016671 0.546089   0.71721097 0.18078542 0.25561773]
 [0.76169627 0.70054323 0.05964359 0.48047568 0.29472567]
 [0.45905745 0.00661603 0.50734427 0.98020438 0.76287948]]

###	NETWORK ARCHITECTURE
Weights Hidden Layer Shape (5, 4)
Weights Output Layer Shape (3, 5)

###	TRAINING NETWORK - ADJUST WEIGHTS
Train Inputs Shape (104, 4)
Train Outputs Shape (104, 3)
Testing will proceed with 20 ephocs!!
	EPOCH: 1
	EPOCH: 2
	EPOCH: 3
	EPOCH: 4
	EPOCH: 5
	EPOCH: 6
	EPOCH: 7
	EPOCH: 8
	EPOCH: 9
	EPOCH: 10
	EPOCH: 11
	EPOCH: 12
	EPOCH: 13
	EPOCH: 14
	EPOCH: 15
	EPOCH: 16
	EPOCH: 17
	EPOCH: 18
	EPOCH: 19
	EPOCH: 20


In [19]:
print('\n###\tTESTING RESULTS')
print(f'Test Inputs Shape {inputs_test.shape}')
print(f'Test Outputs Shape {outputs_test.shape}')

# Calculate results after training
print('\n###\tRAW RESULTS')
results = feed_forward(whl, wol, inputs_test, outputs_test)
results_final = format_results(results)



###	TESTING RESULTS
Test Inputs Shape (45, 4)
Test Outputs Shape (45, 3)

###	RAW RESULTS
d = [0 1 0] y = [0.38620425 0.60294768 0.00496314]
d = [0 0 1] y = [0.06569951 0.00794721 0.95550377]
d = [1 0 0] y = [0.72352854 0.22091705 0.02392526]
d = [1 0 0] y = [0.87349001 0.07567519 0.07679022]
d = [1 0 0] y = [0.73929414 0.20642432 0.02581478]
d = [0 1 0] y = [0.37927025 0.61119908 0.00480377]
d = [0 0 1] y = [0.06131101 0.00765625 0.95914265]
d = [0 1 0] y = [0.37985257 0.61049157 0.0048171 ]
d = [0 1 0] y = [0.37935347 0.61108389 0.00480594]
d = [0 1 0] y = [0.38230193 0.60757474 0.00487351]
d = [1 0 0] y = [0.38367699 0.60593588 0.00490583]
d = [0 1 0] y = [0.39843194 0.58831407 0.00525732]
d = [1 0 0] y = [0.85610285 0.09673994 0.05829536]
d = [0 0 1] y = [0.06145878 0.00768372 0.95902432]
d = [1 0 0] y = [0.85666674 0.09662165 0.05825562]
d = [1 0 0] y = [0.71922903 0.22542643 0.02334291]
d = [0 0 1] y = [0.06091016 0.00764037 0.95947304]
d = [1 0 0] y = [0.87673898 0.08090244 0.0

In [20]:
print('\n###\tFORMATTED RESULTS')
print('EXPECTED', "\t", 'PREDICTED')
for i in range(0, results_final.shape[0]):
    print(outputs_test[i], "\t", results_final[i])


accuracy = accuracy_score(outputs_test,results_final)
print(f'\n### ACCURACY SCORE: {accuracy}')


###	FORMATTED RESULTS
EXPECTED 	 PREDICTED
[0 1 0] 	 [0. 1. 0.]
[0 0 1] 	 [0. 0. 1.]
[1 0 0] 	 [1. 0. 0.]
[1 0 0] 	 [1. 0. 0.]
[1 0 0] 	 [1. 0. 0.]
[0 1 0] 	 [0. 1. 0.]
[0 0 1] 	 [0. 0. 1.]
[0 1 0] 	 [0. 1. 0.]
[0 1 0] 	 [0. 1. 0.]
[0 1 0] 	 [0. 1. 0.]
[1 0 0] 	 [0. 1. 0.]
[0 1 0] 	 [0. 1. 0.]
[1 0 0] 	 [1. 0. 0.]
[0 0 1] 	 [0. 0. 1.]
[1 0 0] 	 [1. 0. 0.]
[1 0 0] 	 [1. 0. 0.]
[0 0 1] 	 [0. 0. 1.]
[1 0 0] 	 [1. 0. 0.]
[0 0 1] 	 [0. 0. 1.]
[1 0 0] 	 [1. 0. 0.]
[1 0 0] 	 [1. 0. 0.]
[0 0 1] 	 [0. 0. 1.]
[0 0 1] 	 [0. 0. 1.]
[0 1 0] 	 [0. 1. 0.]
[0 1 0] 	 [0. 1. 0.]
[0 0 1] 	 [0. 0. 1.]
[0 1 0] 	 [0. 1. 0.]
[0 0 1] 	 [0. 0. 1.]
[1 0 0] 	 [0. 1. 0.]
[0 0 1] 	 [0. 0. 1.]
[0 1 0] 	 [0. 1. 0.]
[0 1 0] 	 [0. 1. 0.]
[0 0 1] 	 [0. 0. 1.]
[0 0 1] 	 [0. 0. 1.]
[0 1 0] 	 [0. 1. 0.]
[1 0 0] 	 [1. 0. 0.]
[1 0 0] 	 [1. 0. 0.]
[1 0 0] 	 [1. 0. 0.]
[0 1 0] 	 [0. 1. 0.]
[0 1 0] 	 [0. 1. 0.]
[1 0 0] 	 [1. 0. 0.]
[0 0 1] 	 [0. 0. 1.]
[0 1 0] 	 [0. 1. 0.]
[0 0 1] 	 [0. 0. 1.]
[0 1 0] 	 [0. 1. 0.]

### ACCURA